# Machine Learning: Getting Density for SAFT-$\gamma$ Mie and Beyond

Now let's turn to predicting density with ANN or other machine learning techniques, trying in parallel development to use 1) SAFT-$\gamma$ Mie parameters, 2) experimental data and loading from API and 3) potentially using atomistic model / forcefield. Here, we will consider using SMILES to read atomistic data as well.

## SAFT-$\gamma$ Mie

In [2]:
import saftgmie as saft
import numpy as np
import pandas as pd
import math
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import svm
from sklearn import gaussian_process
import scipy
from scipy.optimize import fsolve, least_squares, leastsq

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from sklearn import preprocessing

For each case, the key difference in predicting with machine learning algorithm is the input parameters, i.e. how do we represent the molecules when predicting it. So for some typical examples used in previous papers include using molecular weight to predict boiling point, or reduced temperature to predict Boyle temperature etc. For each individual case, we have to be sure of what we are predicting and what we are predicting it with.

For the SAFT-$\gamma$ Mie case, we are predicting the result of molecular simulation and not the physical properties of real fluid per se. There is a difference here that essentially allows me to fit parameters based on the predicted physical properties (i.e. according to experimental data regression of viscosity, these SAFT-$\gamma$ Mie parameters best represent this molecule) which could lead to using different parameterisation for different molecular simulation purposes.

Fitting experimental data is different in the way that we are removing this molecular simulation bridge, essentially using whatever input parameters as the best description available for the molecules. Hence, we can predict the actual fluid properties if we can accurately describe the molecules in the way that we want.

Atomistic model may be more comprehensive that the SAFT-$\gamma$ Mie case, but it is still a matter of fitting simulation data as output, which really depends on the forcefield we are using. To make this into an actual predictive algorithm, what we really need some form of correction towards experimental data. Do take note of any presence of biases towards certain type of molecules (e.g. there are more available info about alkanes as compared to ether)

### Preparing and preprocessing of data

## Experimental Data Fitting

## Atomistic Proof of Concept